In [1]:
import pandas as pd

df = pd.read_csv("../data/02_intermediate/data.csv", dtype={
    'id': 'int64',
    'pdf': 'string',
    'text': 'string'},
                na_filter = False)

In [2]:
df = pd.read_parquet("../data/04_feature/chunks.parquet")

In [3]:
print(df.count())

id       109516
page     109516
chunk    109516
dtype: int64


In [34]:
df.dtypes

id       int64
pdf     string
text    string
dtype: object

In [37]:
df = df[(df["pdf"] != "N/A") & (df["text"] != "N/A")]
df[df["pdf"] == 'N/A']

,id,pdf,text


In [ ]:
import json
import os
import pandas as pd
from pathlib import Path

def get_metadata():
    metadata_path = Path("../data/01_raw/ntrs-public-metadata.json")
    with open(metadata_path, 'r') as f:
        metadata = json.load(f)
    return metadata

def get_metadata_df():
    metadata = get_metadata()
    df = pd.DataFrame(metadata)
    return df

def get_metadata_df_by_year(year):
    df = get_metadata_df()
    df = df[df['year'] == year]
    return df

meta = get_metadata()


In [ ]:
print(meta['19930091025'])

In [ ]:
meta['19930091026']

In [ ]:
import requests as r
from icecream import ic
from collections import defaultdict
import random
random.seed(4)
import time
import numpy as np


BASE_URL = "https://ntrs.nasa.gov/api/citations/"
aval = 0
not_aval = 0
pdf = 0
text = 0
i = 0
counts = defaultdict(int)
print(len(meta.keys()))

def make_request(url):
    # try:
    res = r.request("GET", url)
    while res.status_code != 200:
        time.sleep(np.random.randint(1, 30))
        res = r.request("GET", url)
    # except Exception as e:
        # print(e)
        # return None
    return res

keys = list(meta.keys())
random.shuffle(keys)
keys = keys[:10]
urls = [BASE_URL + key for key in keys]
# urls = [BASE_URL + key + '/download' for key in keys]
# from multiprocessing import Pool
# pool = Pool(4)
# results = pool.map(make_request, urls)
results = [make_request(url) for url in urls]
print(len(results))

texts = []
pdfs = []
fals = 0

for result in results:
    # print(result)
    # next
    if result.status_code != 200:
        # print("429")
        # print("Falure")
        print(result.status_code)
        # print(result)
        fals += 1
        continue

    data = result.json()
    # print()
    # print(data)
    # next

    # print(data.keys())
    # print(data['downloads'])
    # print(data['downloads'])
    counts[len(data['downloads'])] += 1
    if data['downloadsAvailable']:
        # print(data.keys())
        if len(data['downloads']) > 0:
            links = data['downloads'][0]['links']
            if 'pdf' in links:
                pdfs.append(links['pdf'])

            if "fulltext" in links:
                texts.append(links['fulltext'])
            aval += 1
        else:
            print('error')
            # print(data)
            not_aval += 1
    else:
        not_aval += 1
    i+=1
    if i > 1000:
        break
    

print(f"Available: {aval}")
print(f"Not Available: {not_aval}")
print(f"PDF: {len(pdfs)}")
print(f"Text: {len(texts)}")
print(f"Fals: {fals}")
print(counts)


In [ ]:
pdfs[:4]

In [ ]:
for pdf in pdfs:
    

In [ ]:
import requests as r
from icecream import ic
from collections import defaultdict
import random
random.seed(4)


BASE_URL = "https://ntrs.nasa.gov/api/citations/"
aval = 0
not_aval = 0
pdf = 0
text = 0
i = 0
counts = defaultdict(int)
print(len(meta.keys()))

keys = list(meta.keys())
random.shuffle(keys)
for key in keys:
    url = BASE_URL + key + "/downloads"
    result = r.request("GET", url)
    # print(result.content)
    data = result.json()
    # print()
    print(data)
    break
    # print(data['downloads'])
    # print(data['downloads'])
    counts[len(data['downloads'])] += 1
    if data['downloadsAvailable']:
        # print(data.keys())
        try:
            links = data['downloads'][0]['links']
            if links['pdf']:
                pdf += 1
            if links['fulltext']:
                text += 1
            aval += 1
        except:
            print('error')
            # print(data)
            not_aval += 1
    else:
        not_aval += 1
    i+=1
    if i > 1000:
        break
    

print(f"Available: {aval}")
print(f"Not Available: {not_aval}")
print(f"PDF: {pdf}")
print(f"Text: {text}")
print(counts)